In [125]:
import pandas as pd
import numpy as np
import datetime
import json
from haversine import haversine
import zipcodes
import googlemaps
import os
import zipcodes
import paramiko
import logging

logging.basicConfig(filename='SK_weekly_celery.log', level=logging.INFO)

zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/center_of_rentrak_zip.json","r"))

output_path_remote="/home/ruoyi/sk_ta_info/"

my_output_folder_internal="/home/jian/Projects/Smoothie_King/weekly_TA_updates/"



In [126]:
today_date_Monday=datetime.datetime.now().date()
str_8_digits=str(today_date_Monday)[:4]+str(today_date_Monday)[5:7]+str(today_date_Monday)[8:10]
str_8_digits="20190211"

In [127]:
my_output_folder_internal=my_output_folder_internal+"output_"+str_8_digits+"/"
try:
    os.stat(my_output_folder_internal)
except:
    os.mkdir(my_output_folder_internal)

In [3]:
# SK Client Access

host = "64.237.51.251" 
port = 22
transport = paramiko.Transport((host, port))

account_SK="SmoothieKing"
pwd_SK="j$uY_e~23#[:9Zz6"

transport.connect(username = account_SK, password = pwd_SK)
sftp = paramiko.SFTPClient.from_transport(transport)

In [4]:
# email_meassage="Files received: "

new_file_list=sftp.listdir("/mnt/drv5/SmoothieKing")
# new_weekly_store_list=["/home/client/BigLotsData/"+x for x in new_weekly_file_list if Client_Today_NUM_STR in x]
new_weekly_store_list=[x for x in new_file_list if (str_8_digits in x) & ("StoreList" in x)]

if len(new_weekly_store_list)==0:
    print("Store_list_not_received,"+str(datetime.datetime.now()))
elif len(new_weekly_store_list)>1:
    print("Store_list_More_Than_1: "+str(datetime.datetime.now()))
else:
    # email_meassage=email_meassage+str(new_weekly_store_list[0])
    print("Store_list_received")
    store_list_file=new_weekly_store_list[0]
    remote_path="/mnt/drv5/SmoothieKing/"+new_weekly_store_list[0]
    local_path_storelist='/home/jian/Projects/Smoothie_King/store_list_weekly/'+store_list_file
    sftp.get(remote_path,local_path_storelist)
    
    
new_weekly_sales=[x for x in new_file_list if (str_8_digits in x) & ("WeeklySalesSummaryData" in x)]

if len(new_weekly_sales)==0:
    print("Store_WeeklySales_not_received,"+str(datetime.datetime.now()))
elif len(new_weekly_sales)>1:
    print("Store_WeeklySales_More_Than_1: "+str(datetime.datetime.now()))
else:
    # email_meassage=email_meassage+" | "+str(new_weekly_store_list[0])
    print("Store_WeeklySales_received")
    store_list_file=new_weekly_sales[0]
    remote_path="/mnt/drv5/SmoothieKing/"+new_weekly_sales[0]
    local_path_sales='/home/jian/Projects/Smoothie_King/Weekly_Sales/'+new_weekly_sales[0]
    sftp.get(remote_path,local_path_sales)
    
sftp.close()

Store_list_received
Store_WeeklySales_received


In [7]:
import smtplib

from string import Template

from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

MY_ADDRESS = 'jubapluscc@gmail.com'
PASSWORD = 'jubaplus2015'

def get_contacts(filename):
    """
    Return two lists names, emails containing names and email addresses
    read from a file specified by filename.
    """
    
    names = []
    emails = []
    with open(filename, mode='r', encoding='utf-8') as contacts_file:
        for a_contact in contacts_file:
            names.append(a_contact.split()[0])
            emails.append(a_contact.split()[1])
    return names, emails

def read_template(filename):
    """
    Returns a Template object comprising the contents of the 
    file specified by filename.
    """
    
    with open(filename, 'r', encoding='utf-8') as template_file:
        template_file_content = template_file.read()
    return Template(template_file_content)



In [9]:
if (len(new_weekly_sales)==1) & (len(new_weekly_sales)==1):

    names, emails = get_contacts('/home/jian/Projects/Smoothie_King/SK_celery/mycontacts_success.txt') # read contacts
    message_template = read_template('/home/jian/Projects/Smoothie_King/SK_celery/message_success.txt')
    print(names, emails, message_template)
    # set up the SMTP server
    s = smtplib.SMTP(host='smtp.gmail.com', port=587)
    s.starttls()
    s.login(MY_ADDRESS, PASSWORD)

    # For each contact, send the email:
    for name, email in zip(names, emails):
        msg = MIMEMultipart()       # create a message

        # add in the actual person name to the message template
        message = message_template.substitute(PERSON_NAME=name.title())

        # Prints out the message body for our sake
        print(message)

        # setup the parameters of the message
        msg['From']=MY_ADDRESS
        msg['To']=email
        msg['Subject']="Smoothie King Data Received"

        # add in the message body
        msg.attach(MIMEText(message, 'plain'))

        # send the message via the server set up earlier.
        s.send_message(msg)
        del msg

    # Terminate the SMTP session and close the connection
    s.close()

else:
    names, emails = get_contacts('/home/jian/Projects/Smoothie_King/SK_celery/mycontacts_failed.txt') # read contacts
    message_template = read_template('/home/jian/Projects/Smoothie_King/SK_celery/message_failed.txt')
    print(names, emails, message_template)
    # set up the SMTP server
    s = smtplib.SMTP(host='smtp.gmail.com', port=587)
    s.starttls()
    s.login(MY_ADDRESS, PASSWORD)

    # For each contact, send the email:
    for name, email in zip(names, emails):
        msg = MIMEMultipart()       # create a message

        # add in the actual person name to the message template
        message = message_template.substitute(PERSON_NAME=name.title())

        # Prints out the message body for our sake
        print(message)

        # setup the parameters of the message
        msg['From']=MY_ADDRESS
        msg['To']=email
        msg['Subject']="Smoothie King Data Not Received yet"

        # add in the message body
        msg.attach(MIMEText(message, 'plain'))

        # send the message via the server set up earlier.
        s.send_message(msg)
        del msg

    # Terminate the SMTP session and close the connection
    s.close()

['rgao', 'dhys', 'Jian'] ['rgao@jubaplus.com', 'dhysty90@gmail.com', 'jian@jubaplus.com'] <string.Template object at 0x7ff406706c50>
Hi,

The 2 weekly files are recieved from SK.

Thanks
Hi,

The 2 weekly files are recieved from SK.

Thanks
Hi,

The 2 weekly files are recieved from SK.

Thanks


In [5]:
local_path_storelist

'/home/jian/Projects/Smoothie_King/store_list_weekly/StoreList_20190211.xlsx'

In [6]:
store_list_this_week=pd.read_excel(local_path_storelist,dtype=str)
store_list_this_week=store_list_this_week[~pd.isnull(store_list_this_week['storenumber'])]
store_list_this_week=store_list_this_week[store_list_this_week['storenumber']!="nan"]
store_list_this_week=store_list_this_week[store_list_this_week['status']!="Closed"]
store_list_this_week=store_list_this_week.drop_duplicates()
print(str(store_list_this_week['status'].unique().tolist()))

print("Make Sure 0 as storenumber length>4 (StoreList):",str(len(store_list_this_week[store_list_this_week['storenumber'].apply(lambda x: len(x))!=4])))

store_list_this_week=store_list_this_week.reset_index()
del store_list_this_week['index']

print(len(store_list_this_week['storenumber'].unique().tolist())==len(store_list_this_week))
# Filter later

['Same', 'NonComp', 'New', 'Noncomp']
Make Sure 0 as storenumber length>4 (StoreList): 0
True


In [7]:

matched_df_with_SG=pd.read_excel("/home/jian/Projects/Smoothie_King/TA/Celery_SK_TA/QC_Address_With_SG_JL_2019-02-12.xlsx",sheetname="Matched_Stores",dtype=str)
matched_df_with_SG['SG_zip']=matched_df_with_SG['address_comb_SG'].apply(lambda x: x.split("|")[len(x.split("|"))-1])
store_zip_dic_updated=matched_df_with_SG.set_index("storenumber").to_dict()['SG_zip']

store_zip_dic_updated.update({"1152":"70810"})

store_zip_dic_received=store_list_this_week.set_index(['storenumber']).to_dict()['zip']
def update_zip(x):
    if x in store_zip_dic_updated.keys():
        y=store_zip_dic_updated[x]
    else:
        y=store_zip_dic_received[x]
    return y

store_list_this_week['zip']=store_list_this_week['storenumber'].apply(update_zip)
store_list_this_week['zip']=store_list_this_week['zip'].apply(lambda x: x.split("-")[0])


In [8]:
sales_data_this_week=pd.read_excel(local_path_sales,dtype=str)
sales_data_this_week['NetSalesSmoothie']=sales_data_this_week['NetSalesSmoothie'].astype(float)
sales_data_this_week=sales_data_this_week[sales_data_this_week['NetSalesSmoothie']>0]
print("Make Sure 0 as storenumber length>4 (Sales):",str(len(sales_data_this_week[sales_data_this_week['StoreNumber'].apply(lambda x: len(x))!=4])))
print(len(sales_data_this_week['StoreNumber'].unique().tolist())==len(sales_data_this_week))

Make Sure 0 as storenumber length>4 (Sales): 0
True


In [9]:
store_list_this_week=store_list_this_week[store_list_this_week['storenumber'].isin(sales_data_this_week['StoreNumber'].unique().tolist())]
store_list_this_week=store_list_this_week.reset_index()
del store_list_this_week['index']
store_list_this_week.shape

(893, 14)

In [94]:
new_store_list=store_list_this_week[~store_list_this_week['storenumber'].isin(store_GPS_verified['storenumber'].tolist())]

if len(new_store_list)>0:
    new_store_list.to_csv(my_output_folder_internal+"new_store_appared.csv",index=False)
else:
    new_store_list.to_csv(my_output_folder_internal+"no_new_store_detected.csv",index=False)
print(new_store_list.shape)

(0, 15)


In [10]:
store_GPS_verified=pd.read_excel("/home/jian/Projects/Smoothie_King/TA/Celery_SK_TA/SK_store_listed_verified_JL_2019-02-12.xlsx",sheetname="assigned_lat_long",dtype=str)
store_GPS_verified['assigned_lat']=store_GPS_verified['assigned_lat'].astype(float)
store_GPS_verified['assigned_long']=store_GPS_verified['assigned_long'].astype(float)
store_GPS_verified['storenumber'].apply(lambda x: len(x)).unique()


array([4])

In [11]:
verified_store_lat_dict=store_GPS_verified.set_index("storenumber").to_dict()['assigned_lat']
verified_store_lng_dict=store_GPS_verified.set_index("storenumber").to_dict()['assigned_long']

store_list_this_week['used_lat']=store_list_this_week['storenumber'].apply(lambda x: float(verified_store_lat_dict[x]))
store_list_this_week['used_long']=store_list_this_week['storenumber'].apply(lambda x: float(verified_store_lng_dict[x]))

# Get 3 miles zips based on lat lng

In [12]:
store_list_this_week=store_list_this_week.sort_values('storenumber').reset_index()
del store_list_this_week['index']
store_list_this_week['zips_3_miles']=np.nan
for i in range(len(store_list_this_week)):
    store_id=store_list_this_week['storenumber'][i]
    store_center=[verified_store_lat_dict[store_id],verified_store_lng_dict[store_id]]
    list_zip_3_miles=[store_list_this_week['zip'][i]]
    for zip_cd in zip_centers.keys():
        dist=haversine(store_center,zip_centers[zip_cd],miles=True)
        if dist<=3:
            list_zip_3_miles=list_zip_3_miles+[zip_cd]
    store_list_this_week['zips_3_miles'][i]=list(set(list_zip_3_miles))
            
data=store_list_this_week.copy()
store_list_df=store_list_this_week.copy()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [14]:
data['TA']=np.nan
data=data.reset_index()
del data['index']
df_TA_zips=pd.DataFrame({"store":[data['storenumber'][0]]*len(data['zips_3_miles'][0]),
                         "zip":data['zips_3_miles'][0],
                         "TA":[1]*len(data['zips_3_miles'][0])},
                        index=[1]*len(data['zips_3_miles'][0]))

In [60]:
df_TA_zips[df_TA_zips['store']=="1117"]

,TA,store,zip
461,16,1117,36616
461,16,1117,36660
461,16,1117,36617
461,16,1117,36606
461,16,1117,36604
461,16,1117,36607
461,16,1117,36670


In [22]:
TA_counter=1

for i in range(1,len(data)):
    intersection_zip=list(set(data['zips_3_miles'][i]).intersection(set(df_TA_zips['zip'].unique().tolist())))
    if len(intersection_zip)==0:
        TA_counter+=1
        df_TA_zips=df_TA_zips.append(pd.DataFrame({"store":[data['storenumber'][i]]*len(data['zips_3_miles'][i]),
                                                   "zip":data['zips_3_miles'][i],
                                                   "TA":[TA_counter]*len(data['zips_3_miles'][i])},
                                                  index=[i]*len(data['zips_3_miles'][i]))).drop_duplicates()
        
    else:
        df_intersection=df_TA_zips[df_TA_zips['zip'].isin(intersection_zip)]
        group_df_intersection=df_intersection.groupby(['TA'])['zip'].count().to_frame().reset_index().sort_values(['zip'],ascending=False)
        selected_TA=group_df_intersection['TA'][0] 
        
        df_TA_zips_0=df_TA_zips[~df_TA_zips['TA'].isin(set(group_df_intersection['TA']))]
        df_TA_zips_1=df_TA_zips[df_TA_zips['TA'].isin(group_df_intersection['TA'].tolist())]
        df_TA_zips_1['TA']=selected_TA
        df_TA_zips=df_TA_zips_0.append(df_TA_zips_1).append(pd.DataFrame({"store":[data['storenumber'][i]]*len(data['zips_3_miles'][i]),
                                                                          "zip":data['zips_3_miles'][i],
                                                                          "TA":[selected_TA]*len(data['zips_3_miles'][i])},
                                                                         index=[i]*len(data['zips_3_miles'][i]))).drop_duplicates()
        
dict_TA_zips=df_TA_zips.set_index('zip').to_dict()['TA']
dict_TA_store=df_TA_zips.set_index('store').to_dict()['TA']
data['TA']=data['storenumber'].apply(lambda x: dict_TA_store[x])

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [23]:
df_ta_num_unique=data[['TA']].drop_duplicates().reset_index()
del df_ta_num_unique['index']
df_ta_num_unique['new_TA']=[x+1 for x in range(len(df_ta_num_unique))]

dict_ta_num_unique=df_ta_num_unique.set_index(['TA']).to_dict()['new_TA']
data['TA']=data['TA'].apply(lambda x: dict_ta_num_unique[x])
df_TA_zips['TA']=df_TA_zips['TA'].apply(lambda x: dict_ta_num_unique[x])


In [35]:
summary_store=data.groupby("TA")['storenumber'].count().to_frame().reset_index().rename(columns={"storenumber":"store_count"})
summary_zip=df_TA_zips[['TA','zip']].drop_duplicates().groupby("TA")['zip'].count().to_frame().reset_index().rename(columns={"zip":"zip_count"})
# summary_store_2=df_TA_zips[['TA','store']].drop_duplicates().groupby("TA")['store'].count().to_frame().reset_index().rename(columns={"store":"store_count_2"})
summary_store_list=data.groupby("TA")['storenumber'].apply(list).to_frame().reset_index().rename(columns={"storenumber":"store_list"})
summary_zip_list=df_TA_zips[['TA','zip']].drop_duplicates().groupby("TA")['zip'].apply(list).to_frame().reset_index().rename(columns={"zip":"zip_list"})



summary_by_TA=pd.merge(summary_store,summary_zip,on="TA",how="outer")
summary_by_TA=pd.merge(summary_by_TA,summary_store_list,on="TA",how="outer")
summary_by_TA=pd.merge(summary_by_TA,summary_zip_list,on="TA",how="outer")

# summary=pd.merge(summary,summary_store_2,on="TA",how="outer")
TA_Store_zip_list=data.groupby(['TA'])['zip'].apply(set).to_frame().reset_index().rename(columns={"zip":"store_zip_list"})
summary_by_TA=pd.merge(summary_by_TA,TA_Store_zip_list,on="TA",how="left")


In [40]:
len(summary_by_TA['store_zip_list'][0])

24

In [42]:
summary_by_store_count=summary_by_TA.groupby(['store_count'])['TA'].count().to_frame().reset_index().rename(columns={"TA":"TA_count"})
summary_by_store_list=summary_by_TA.groupby(['store_count'])['TA'].apply(list).to_frame().reset_index().rename(columns={"TA":"TA_list"})
summary_by_store_count=pd.merge(summary_by_store_count,summary_by_store_list,on="store_count",how="outer")


In [44]:
df_TA_zips.head(5)

,TA,store,zip
8,3,0011,70047
8,3,0011,70087
8,3,0011,70031
12,5,0016,70607
12,5,0016,70606


In [45]:
df_zip_dist_in_TA=df_TA_zips[["TA","zip"]].drop_duplicates().reset_index()
del df_zip_dist_in_TA['index']

output_distance_zip_in_TA=pd.DataFrame()
counter_k=0
for ta,group in df_zip_dist_in_TA.groupby(['TA']):
    group=group.reset_index()
    del group['index']
    
    if len(group)>1:
    
        dist_list=[]

        for i in range(len(group)):
            zip_hold=group['zip'][i]

            if zip_hold not in zip_centers.keys():
                try:
                    zip_hold_center=(zipcodes.matching(zip_hold)[0]['lat'],zipcodes.matching(zip_hold)[0]['long'])
                except:
                    print("zip not found, ",zip_hold)

            else:
                zip_hold_center=zip_centers[zip_hold]

            for j in range(i+1,len(group)):
                zip_var=group['zip'][j]
                if zip_var not in zip_centers.keys():
                    try:
                        zip_var_center=(zipcodes.matching(zip_var)[0]['lat'],zipcodes.matching(zip_var)[0]['long'])
                    except:
                        print("zip not found, ",zip_hold)

                else:
                    zip_var_center=zip_centers[zip_var]

                try:
                    dist=haversine(zip_hold_center,zip_var_center,miles=True)
                except:
                    dist=np.nan

                dist_list=dist_list+[dist]
        df=pd.DataFrame({"TA":ta,"dist_min":min(dist_list),"dist_max":max(dist_list),"dist_median":np.median(dist_list),"All_dist":[dist_list]},index=[counter_k])
        counter_k+=1
        output_distance_zip_in_TA=output_distance_zip_in_TA.append(df)
    else:
        df=pd.DataFrame({"TA":ta,"dist_min":0,"dist_max":0,"dist_median":0,"All_dist":"single_zip"},index=[counter_k])
        output_distance_zip_in_TA=output_distance_zip_in_TA.append(df)
    

In [46]:
zip_DMA=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",dtype=str,skiprows=1)
zip_DMA=zip_DMA.iloc[:,[0,2]]
zip_DMA.columns=['zip','DMA']

In [47]:
df_city_state_in_TA=df_TA_zips[["TA","zip"]].drop_duplicates().reset_index()
del df_city_state_in_TA['index']

def city_of_zip(x):
    try:
        city=zipcodes.matching(x)[0]['city']
    except:
        city=np.nan
    return city

def state_of_zip(x):
    try:
        state=zipcodes.matching(x)[0]['state']
    except:
        state=np.nan
    return state
    
df_city_state_in_TA['city']=df_city_state_in_TA['zip'].apply(lambda x: city_of_zip(x))
df_city_state_in_TA['state']=df_city_state_in_TA['zip'].apply(lambda x: state_of_zip(x))
df_city_state_in_TA['city']=df_city_state_in_TA['city']+" ("+df_city_state_in_TA['state']+")"



In [48]:
df_DMA_in_TA=df_TA_zips[["TA","zip"]].drop_duplicates().reset_index()
del df_DMA_in_TA['index']
df_DMA_in_TA=pd.merge(df_DMA_in_TA,zip_DMA,on="zip",how="left")
df_DMA_in_TA.head(2)

,TA,zip,DMA
0,3,70047,NEW ORLEANS
1,3,70087,NEW ORLEANS


In [49]:
df_city_TA_list=df_city_state_in_TA.groupby(['TA'])['city'].apply(set).to_frame().reset_index()
df_city_TA_list=df_city_TA_list.rename(columns={"city":"city_list"})
df_state_TA_list=df_city_state_in_TA.groupby(['TA'])['state'].apply(set).to_frame().reset_index()
df_state_TA_list=df_state_TA_list.rename(columns={"state":"state_list"})
df_DMA_TA_list=df_DMA_in_TA.groupby(['TA'])['DMA'].apply(set).to_frame().reset_index()
df_DMA_TA_list=df_DMA_TA_list.rename(columns={"DMA":"DMA_list"})


In [50]:
manually_city_dict_of_zip={"29486":"Summerville"}
manually_state_dict_of_zip={"29486":"SC"}
df_city_state_in_TA['city'][df_city_state_in_TA['zip']=='29486']=manually_city_dict_of_zip['29486']
df_city_state_in_TA['state'][df_city_state_in_TA['zip']=='29486']=manually_state_dict_of_zip['29486']

removed_2_zips_df=df_city_state_in_TA[pd.isnull(df_city_state_in_TA['city'])]
df_city_state_in_TA=df_city_state_in_TA[~pd.isnull(df_city_state_in_TA['city'])]
# zip "08644","76190" removed to determine cities/states because of not existing as in https://www.unitedstateszipcodes.org/ 

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [55]:
counter_k=1
df_primary_city_state=pd.DataFrame()
df_primary_DMA=pd.DataFrame()
for ta,group in df_city_state_in_TA.groupby(['TA']):
    df_city=group.groupby(['city'])['zip'].count().to_frame().reset_index().sort_values(['zip'],ascending=False).reset_index()
    del df_city['index']
    primary_city=df_city['city'][0]
    
    df_state=group.groupby(['state'])['zip'].count().to_frame().reset_index().sort_values(['zip'],ascending=False).reset_index()
    del df_state['index']
    primary_state=df_state['state'][0]
    
    df=pd.DataFrame({"TA":ta,"Primary_City":primary_city,"Primary_State":primary_state},index=[counter_k])
    counter_k+=1
    df_primary_city_state=df_primary_city_state.append(df)

    
for ta,group in df_DMA_in_TA.groupby(['TA']):
    df_DMA=group.groupby(['DMA'])['zip'].count().to_frame().reset_index().sort_values(['zip'],ascending=False).reset_index()
    del df_DMA['index']
    primary_DMA=df_DMA['DMA'][0]
    
    df=pd.DataFrame({"TA":ta,"Primary_DMA":primary_DMA},index=[counter_k])
    counter_k+=1
    df_primary_DMA=df_primary_DMA.append(df)

In [56]:
summary_by_TA=pd.merge(summary_by_TA,df_city_TA_list,on="TA",how="left")
summary_by_TA=pd.merge(summary_by_TA,df_state_TA_list,on="TA",how="left")
summary_by_TA=pd.merge(summary_by_TA,df_DMA_TA_list,on="TA",how="left")

summary_by_TA=pd.merge(summary_by_TA,df_primary_city_state,on="TA",how="left")
summary_by_TA=pd.merge(summary_by_TA,df_primary_DMA,on="TA",how="left")

summary_by_TA=pd.merge(summary_by_TA,output_distance_zip_in_TA,on="TA",how="left")

In [57]:
data['Latitude']=data['storenumber'].apply(lambda x: float(verified_store_lat_dict[x]))
data['Longitude']=data['storenumber'].apply(lambda x: float(verified_store_lng_dict[x]))
data['TA']=data['TA'].astype(str)
summary_by_TA['TA']=summary_by_TA['TA'].astype(str)

# Split TA

In [58]:
summary_by_TA['ratio_max_media']=summary_by_TA['dist_max']/summary_by_TA['dist_median']

summary_by_TA_to_revise=summary_by_TA[(summary_by_TA['ratio_max_media']>2) &\
                                      (summary_by_TA['store_count']>=2) &\
                                     (summary_by_TA['dist_max']>12)]
summary_by_TA_to_keep=summary_by_TA[(summary_by_TA['ratio_max_media']<=2) |\
                                      (summary_by_TA['store_count']<2) |\
                                     (summary_by_TA['dist_max']<=12)]
summary_by_TA_to_revise=summary_by_TA_to_revise.reset_index()
del summary_by_TA_to_revise['index']

summary_by_TA_to_keep=summary_by_TA_to_keep.reset_index()
del summary_by_TA_to_keep['index']

In [68]:
loop_counter=0
while len(summary_by_TA_to_revise)>0:
    loop_counter+=1
    if loop_counter>30:
        break
    
    summary_by_TA_to_revise=summary_by_TA_to_revise.reset_index()
    del summary_by_TA_to_revise['index']

    summary_by_TA_to_keep=summary_by_TA_to_keep.reset_index()
    del summary_by_TA_to_keep['index']

    data_keep=data[data['TA'].isin(summary_by_TA_to_keep['TA'])]
    data_revise=data[~data['TA'].isin(summary_by_TA_to_keep['TA'])]
    data_revise=data_revise.reset_index()
    del data_revise['index']


    

    store_sub_df=pd.DataFrame()
    for i in range(len(summary_by_TA_to_revise)):
        TA=summary_by_TA_to_revise['TA'][i]
        store_list=summary_by_TA_to_revise['store_list'][i].copy()
        initial_dist=0
        store_pair=[np.nan,np.nan]
        while len(store_list)>=2:
            store_hold=store_list[0]
            store_list.remove(store_hold)
            store_hold_center=[verified_store_lat_dict[store_hold],verified_store_lng_dict[store_hold]]
            for store_var in store_list:
                store_var_center=[verified_store_lat_dict[store_var],verified_store_lng_dict[store_var]]
                dist=haversine(store_hold_center,store_var_center,miles=True)
                if dist>initial_dist:
                    initial_dist=dist
                    store_pair=[store_hold,store_var]
        store_a=store_pair[0]
        store_b=store_pair[1]

        store_a_center=[verified_store_lat_dict[store_a],verified_store_lng_dict[store_a]]
        store_b_center=[verified_store_lat_dict[store_b],verified_store_lng_dict[store_b]]
        store_list=summary_by_TA_to_revise['store_list'][i].copy()
        for store in store_list:
            store_center=[verified_store_lat_dict[store],verified_store_lng_dict[store]]
            dist_a=haversine(store_a_center,store_center,miles=True)
            dist_b=haversine(store_b_center,store_center,miles=True)
            if dist_a<dist_b:
                sub_group="a"
            else:
                sub_group="b"
            df=pd.DataFrame({"storenumber":store,"TA":TA,"sub_group":sub_group},index=[store])
            store_sub_df=store_sub_df.append(df)
    
    store_sub_df['TA']=store_sub_df['TA'].astype(str)
    store_sub_df['TA']=store_sub_df['TA']+"_"+store_sub_df['sub_group']
    store_sub_df=store_sub_df[['storenumber','TA']]
    store_sub_df_dic=store_sub_df.set_index(['storenumber']).to_dict()['TA']
                    
    data_revise['TA']=data_revise['storenumber'].apply(lambda x: store_sub_df_dic[x])
    data=data_keep.append(data_revise)
    data=data.sort_values(['storenumber'])
    data=data.reset_index()
    del data['index']
    data['TA']=data['TA'].apply(lambda x: str(x).zfill(5))

    df_TA_zips=pd.DataFrame()
    for i in range(len(data)):
        df=pd.DataFrame({"store":[data['storenumber'][i]]*len(data['zips_3_miles'][i]),
                         "TA":[data['TA'][i]]*len(data['zips_3_miles'][i]),
                         "zip":data['zips_3_miles'][i]},index=data['zips_3_miles'][i])
        df_TA_zips=df_TA_zips.append(df)


    summary_store=data.groupby("TA")['storenumber'].count().to_frame().reset_index().rename(columns={"storenumber":"store_count"})
    summary_zip=df_TA_zips[['TA','zip']].drop_duplicates().groupby("TA")['zip'].count().to_frame().reset_index().rename(columns={"zip":"zip_count"})
    summary_store_list=data.groupby("TA")['storenumber'].apply(list).to_frame().reset_index().rename(columns={"storenumber":"store_list"})
    summary_zip_list=df_TA_zips[['TA','zip']].drop_duplicates().groupby("TA")['zip'].apply(list).to_frame().reset_index().rename(columns={"zip":"zip_list"})

    summary_by_TA=pd.merge(summary_store,summary_zip,on="TA",how="outer")
    summary_by_TA=pd.merge(summary_by_TA,summary_store_list,on="TA",how="outer")
    summary_by_TA=pd.merge(summary_by_TA,summary_zip_list,on="TA",how="outer")

    # summary=pd.merge(summary,summary_store_2,on="TA",how="outer")
    TA_Store_zip_list=data.groupby(['TA'])['zip'].apply(set).to_frame().reset_index().rename(columns={"zip":"store_zip_list"})
    summary_by_TA=pd.merge(summary_by_TA,TA_Store_zip_list,on="TA",how="left")


    summary_by_store_count=summary_by_TA.groupby(['store_count'])['TA'].count().to_frame().reset_index().rename(columns={"TA":"TA_count"})
    summary_by_store_list=summary_by_TA.groupby(['store_count'])['TA'].apply(list).to_frame().reset_index().rename(columns={"TA":"TA_list"})
    summary_by_store_count=pd.merge(summary_by_store_count,summary_by_store_list,on="store_count",how="outer")

    df_zip_dist_in_TA=df_TA_zips[["TA","zip"]].drop_duplicates().reset_index()
    del df_zip_dist_in_TA['index']

    output_distance_zip_in_TA=pd.DataFrame()
    counter_k=0
    for ta,group in df_zip_dist_in_TA.groupby(['TA']):
        group=group.reset_index()
        del group['index']

        if len(group)>1:

            dist_list=[]

            for i in range(len(group)):
                zip_hold=group['zip'][i]

                if zip_hold not in zip_centers.keys():
                    try:
                        zip_hold_center=(zipcodes.matching(zip_hold)[0]['lat'],zipcodes.matching(zip_hold)[0]['long'])
                    except:
                        print("zip not found, ",zip_hold)

                else:
                    zip_hold_center=zip_centers[zip_hold]

                for j in range(i+1,len(group)):
                    zip_var=group['zip'][j]
                    if zip_var not in zip_centers.keys():
                        try:
                            zip_var_center=(zipcodes.matching(zip_var)[0]['lat'],zipcodes.matching(zip_var)[0]['long'])
                        except:
                            print("zip not found, ",zip_hold)

                    else:
                        zip_var_center=zip_centers[zip_var]

                    try:
                        dist=haversine(zip_hold_center,zip_var_center,miles=True)
                    except:
                        dist=np.nan

                    dist_list=dist_list+[dist]
            df=pd.DataFrame({"TA":ta,"dist_min":min(dist_list),"dist_max":max(dist_list),"dist_median":np.median(dist_list),"All_dist":[dist_list]},index=[counter_k])
            counter_k+=1
            output_distance_zip_in_TA=output_distance_zip_in_TA.append(df)
        else:
            df=pd.DataFrame({"TA":ta,"dist_min":0,"dist_max":0,"dist_median":0,"All_dist":"single_zip"},index=[counter_k])
            output_distance_zip_in_TA=output_distance_zip_in_TA.append(df)

    zip_DMA=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",dtype=str,skiprows=1)
    zip_DMA=zip_DMA.iloc[:,[0,2]]
    zip_DMA.columns=['zip','DMA']

    df_city_state_in_TA=df_TA_zips[["TA","zip"]].drop_duplicates().reset_index()
    del df_city_state_in_TA['index']

    def city_of_zip(x):
        try:
            city=zipcodes.matching(x)[0]['city']
        except:
            city=np.nan
        return city

    def state_of_zip(x):
        try:
            state=zipcodes.matching(x)[0]['state']
        except:
            state=np.nan
        return state

    df_city_state_in_TA['city']=df_city_state_in_TA['zip'].apply(lambda x: city_of_zip(x))
    df_city_state_in_TA['state']=df_city_state_in_TA['zip'].apply(lambda x: state_of_zip(x))
    
    manually_city_dict_of_zip={"29486":"Summerville"}
    manually_state_dict_of_zip={"29486":"SC"}
    df_city_state_in_TA['city'][df_city_state_in_TA['zip']=='29486']=manually_city_dict_of_zip['29486']
    df_city_state_in_TA['state'][df_city_state_in_TA['zip']=='29486']=manually_state_dict_of_zip['29486']

    removed_2_zips_df=df_city_state_in_TA[pd.isnull(df_city_state_in_TA['city'])]
    df_city_state_in_TA=df_city_state_in_TA[~pd.isnull(df_city_state_in_TA['city'])]
    
    df_city_state_in_TA['city']=df_city_state_in_TA['city']+" ("+df_city_state_in_TA['state']+")"

    df_DMA_in_TA=df_TA_zips[["TA","zip"]].drop_duplicates().reset_index()
    del df_DMA_in_TA['index']
    df_DMA_in_TA=pd.merge(df_DMA_in_TA,zip_DMA,on="zip",how="left")

    df_city_TA_list=df_city_state_in_TA.groupby(['TA'])['city'].apply(set).to_frame().reset_index()
    df_city_TA_list=df_city_TA_list.rename(columns={"city":"city_list"})
    df_state_TA_list=df_city_state_in_TA.groupby(['TA'])['state'].apply(set).to_frame().reset_index()
    df_state_TA_list=df_state_TA_list.rename(columns={"state":"state_list"})
    df_DMA_TA_list=df_DMA_in_TA.groupby(['TA'])['DMA'].apply(set).to_frame().reset_index()
    df_DMA_TA_list=df_DMA_TA_list.rename(columns={"DMA":"DMA_list"})

    counter_k
    df_primary_city_state=pd.DataFrame()
    df_primary_DMA=pd.DataFrame()
    
    df_city_state_in_TA
    for ta,group in df_city_state_in_TA.groupby(['TA']):
        df_city=group.groupby(['city'])['zip'].count().to_frame().reset_index().sort_values(['zip'],ascending=False).reset_index()
        del df_city['index']
        primary_city=df_city['city'][0]

        df_state=group.groupby(['state'])['zip'].count().to_frame().reset_index().sort_values(['zip'],ascending=False).reset_index()
        del df_state['index']
        primary_state=df_state['state'][0]

        df=pd.DataFrame({"TA":ta,"Primary_City":primary_city,"Primary_State":primary_state},index=[counter_k])
        counter_k+=1
        df_primary_city_state=df_primary_city_state.append(df)


    for ta,group in df_DMA_in_TA.groupby(['TA']):
        df_DMA=group.groupby(['DMA'])['zip'].count().to_frame().reset_index().sort_values(['zip'],ascending=False).reset_index()
        del df_DMA['index']
        primary_DMA=df_DMA['DMA'][0]

        df=pd.DataFrame({"TA":ta,"Primary_DMA":primary_DMA},index=[counter_k])
        counter_k+=1
        df_primary_DMA=df_primary_DMA.append(df)

    summary_by_TA=pd.merge(summary_by_TA,df_city_TA_list,on="TA",how="left")
    summary_by_TA=pd.merge(summary_by_TA,df_state_TA_list,on="TA",how="left")
    summary_by_TA=pd.merge(summary_by_TA,df_DMA_TA_list,on="TA",how="left")

    summary_by_TA=pd.merge(summary_by_TA,df_primary_city_state,on="TA",how="left")
    summary_by_TA=pd.merge(summary_by_TA,df_primary_DMA,on="TA",how="left")

    summary_by_TA=pd.merge(summary_by_TA,output_distance_zip_in_TA,on="TA",how="left")
    summary_by_TA['Initial_TA']=summary_by_TA['TA'].apply(lambda x: int(x.split("_")[0]))
    summary_by_TA=summary_by_TA.sort_values(['Initial_TA','TA'])

    summary_by_TA['ratio_max_media']=summary_by_TA['dist_max']/summary_by_TA['dist_median']

    summary_by_TA_to_revise=summary_by_TA[(summary_by_TA['ratio_max_media']>2) &\
                                          (summary_by_TA['store_count']>=2) &\
                                         (summary_by_TA['dist_max']>12)]
    summary_by_TA_to_keep=summary_by_TA[(summary_by_TA['ratio_max_media']<=2) |\
                                          (summary_by_TA['store_count']<2) |\
                                         (summary_by_TA['dist_max']<=12)]

In [133]:
summary_by_TA

,TA,store_count,zip_count,store_list,zip_list,zip,city_list,state_list,DMA_list,Primary_City,Primary_State,Primary_DMA,All_dist,dist_max,dist_median,dist_min,Initial_TA,ratio_max_media
179,001_a_a,16,46,"[0005, 0007, 0015, 0023, 0025, 0157, 0338, 035...","[70096, 70175, 70125, 70113, 70115, 70073, 701...","{70056, 70053, 70130, 70125, 70139, 70119, 701...","{HARVEY (LA), WESTWEGO (LA), NEW ORLEANS (LA),...",{LA},{NEW ORLEANS},NEW ORLEANS (LA),LA,NEW ORLEANS,"[1.506283930235398, 2.8115701806039444, 2.4270...",10.766885,3.709204,0.000000e+00,1,2.902748
180,001_a_b,3,11,"[0004, 0020, 0031]","[70126, 70186, 70148, 70122, 70182, 70075, 700...","{70127, 70122, 70043}","{CHALMETTE (LA), NEW ORLEANS (LA), MERAUX (LA)...",{LA},{NEW ORLEANS},NEW ORLEANS (LA),LA,NEW ORLEANS,"[0.5318625486155398, 2.7833658048750176, 2.731...",10.853224,5.315580,5.318625e-01,1,2.041776
181,001_b,13,26,"[0002, 0003, 0006, 0010, 0014, 0027, 0217, 027...","[70003, 70001, 70009, 70010, 70006, 70183, 700...","{70003, 70123, 70001, 70124, 70006, 70121, 700...","{KENNER (LA), METAIRIE (LA), WESTWEGO (LA), NE...",{LA},{NEW ORLEANS},METAIRIE (LA),LA,NEW ORLEANS,"[2.7754441632485736, 3.458130344004741, 3.4581...",10.139228,3.862778,0.000000e+00,1,2.624854
0,00002,4,4,"[0009, 0106, 0495, 1427]","[70461, 70458, 70459, 70460]","{70461, 70458}",{SLIDELL (LA)},{LA},{NEW ORLEANS},SLIDELL (LA),LA,NEW ORLEANS,"[2.8524502713546913, 3.070072259514937, 5.4072...",5.407243,2.897932,4.620086e-01,2,1.865897
1,00003,1,3,[0011],"[70047, 70087, 70031]",{70047},"{DESTREHAN (LA), SAINT ROSE (LA), AMA (LA)}",{LA},{NEW ORLEANS},AMA (LA),LA,NEW ORLEANS,"[2.601662226470203, 1.3813418103836035, 1.3626...",2.601662,1.381342,1.362618e+00,3,1.883431
2,00004,3,4,"[0013, 1022, 1292]","[70403, 70401, 70402, 70404]","{70401, 70403}",{HAMMOND (LA)},{LA},"{NEW ORLEANS, BATON ROUGE}",HAMMOND (LA),LA,NEW ORLEANS,"[2.4313963703431125, 1.9850587497137255, 1.616...",2.431396,1.439454,4.465261e-01,4,1.689110
3,00005,1,3,[0016],"[70607, 70606, 70605]",{70605},{LAKE CHARLES (LA)},{LA},{LAKE CHARLES},LAKE CHARLES (LA),LA,LAKE CHARLES,"[2.5599439480680384, 3.3278958538383074, 0.861...",3.327896,2.559944,8.611177e-01,5,1.299988
4,00006,3,2,"[0022, 1104, 1257]","[70448, 70471]","{70448, 70471}",{MANDEVILLE (LA)},{LA},{NEW ORLEANS},MANDEVILLE (LA),LA,NEW ORLEANS,[2.6761204549639053],2.676120,2.676120,2.676120e+00,6,1.000000
493,007_b_b_b_b,2,6,"[0124, 0169]","[77042, 77080, 77092, 77243, 77280, 77040]","{77040, 77042}",{HOUSTON (TX)},{TX},{HOUSTON},HOUSTON (TX),TX,HOUSTON,"[5.761063941311653, 8.054411186217404, 5.79392...",9.421244,3.896999,7.626666e-13,7,2.417564
492,007_b_b_b_a_b_b,1,7,[0344],"[77043, 77080, 77224, 77243, 77280, 77024, 77279]",{77024},{HOUSTON (TX)},{TX},{HOUSTON},HOUSTON (TX),TX,HOUSTON,"[2.568555633887345, 1.5395766331487433, 2.8185...",3.590295,2.818573,0.000000e+00,7,1.273799


In [70]:
summary_by_TA_output=pd.DataFrame()
summary_by_TA=summary_by_TA.sort_values(['Initial_TA'])
for old_ta,group in summary_by_TA.groupby(['Initial_TA']):
    if len(group)>1:
        group['TA']=[str(old_ta).zfill(3)+"_"+str(x+1) for x in range(len(group))]
    else:
        group['TA']=str(old_ta).zfill(3)
    summary_by_TA_output=summary_by_TA_output.append(group)
ta_store_list_dic=summary_by_TA_output.set_index(['TA']).to_dict()['store_list']

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [71]:
store_to_new_ta_dict={}
for new_ta in ta_store_list_dic.keys():
    length=len(ta_store_list_dic[new_ta])
    df=pd.DataFrame({"store":ta_store_list_dic[new_ta],"new_ta":[new_ta]*length},index=[x for x in range(length)])
    df_dict=df.set_index(['store']).to_dict()['new_ta']
    store_to_new_ta_dict.update(df_dict)
len(store_to_new_ta_dict)    
data['TA']=data['storenumber'].apply(lambda x: store_to_new_ta_dict[x])
df_TA_zips['TA']=df_TA_zips['store'].apply(lambda x: store_to_new_ta_dict[x])

summary_by_store_count=summary_by_TA_output.groupby(['store_count'])['TA'].count().to_frame().reset_index().rename(columns={"TA":"TA_count"})
summary_by_store_list=summary_by_TA_output.groupby(['store_count'])['TA'].apply(list).to_frame().reset_index().rename(columns={"TA":"TA_list"})
summary_by_store_count=pd.merge(summary_by_store_count,summary_by_store_list,on="store_count",how="outer")

In [73]:
data_for_tableau=data[['storenumber','zip','city','state','Latitude','Longitude','TA']]
for col in data_for_tableau.columns.tolist()[1:6]:
    data_for_tableau=data_for_tableau.rename(columns={col:"store_"+col})
data_for_tableau=data_for_tableau.append(df_TA_zips[['TA','zip']])
data_for_tableau['store_Latitude']=data_for_tableau['store_Latitude'].fillna(0)
data_for_tableau['store_Longitude']=data_for_tableau['store_Longitude'].fillna(0)

In [137]:
summary_by_TA_output=summary_by_TA_output.reset_index()
del summary_by_TA_output['index']
summary_by_TA_output['latest_TA']=np.nan
for i in range(len(summary_by_TA_output)):
    store_count_str=str(summary_by_TA_output['store_count'][i])
    store_list_str=str(summary_by_TA_output['store_list'][i])
    summary_by_TA_output['latest_TA'][i]=store_count_str+"_"+store_list_str

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [146]:
TA_rename_dict=summary_by_TA_output.set_index("TA").to_dict()['latest_TA']

summary_by_TA_output['TA']=summary_by_TA_output['TA'].apply(lambda x: TA_rename_dict[x])
data['TA']=data['TA'].apply(lambda x: TA_rename_dict[x])
df_TA_zips['TA']=df_TA_zips['TA'].apply(lambda x: TA_rename_dict[x])

KeyError: '001_3'

In [97]:
writer=pd.ExcelWriter(my_output_folder_internal+"SmoothieKing_TA_of_3_miles_zips_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
summary_by_TA_output.to_excel(writer,"summary_by_TA",index=False)
data=data.sort_values(['storenumber'])
data.to_excel(writer,"output_TA_by_store",index=False)
df_TA_zips.to_excel(writer,"zip_TA",index=False)
# data_for_tableau.to_excel(writer,"data_for_tableau",index=False)

del summary_by_store_count['TA_list']
summary_by_store_count.to_excel(writer,"summary_by_store_count",index=False)

writer.save()

In [144]:
df_TA_zips

,TA,store,zip
70003,001_3,0002,70003
70001,001_3,0002,70001
70009,001_3,0002,70009
70010,001_3,0002,70010
70006,001_3,0002,70006
70183,001_3,0002,70183
70060,001_3,0002,70060
70002,001_3,0002,70002
70181,001_3,0002,70181
70005,001_3,0002,70005


In [128]:
# Copy to Ruoyi

# SK Client Access

host = "64.237.51.251" 
port = 22
transport = paramiko.Transport((host, port))

myaccount="jian"
pwd_mine="jian@juba2017"

transport.connect(username = myaccount, password = pwd_mine)
sftp = paramiko.SFTPClient.from_transport(transport)


In [129]:
remote_path_to_put=output_path_remote+"SmoothieKing_TA_of_3_miles_zips_JL_"+str(datetime.datetime.now().date())+".xlsx"
my_file_location=my_output_folder_internal+"SmoothieKing_TA_of_3_miles_zips_JL_"+str(datetime.datetime.now().date())+".xlsx"
sftp.put(my_file_location,remote_path_to_put)
sftp.close()

In [130]:
output_path_remote

'/home/ruoyi/sk_ta_info/'